In [19]:
import pandas as pd
import gzip
from collections import defaultdict
from scipy.stats import mannwhitneyu
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
import numpy as np

In [22]:
# === 경로 설정 ===
df_path = "/Users/jeongmin/Downloads/df_filtered.csv"
idmap_path = "/Users/jeongmin/Downloads/ensembl_mouse_mapping.tsv"
goa_path = "/Users/jeongmin/Downloads/goa_mouse_filtered.gaf"

In [25]:
# === Step 1: 사용자 데이터 불러오기 ===
df_filtered = pd.read_csv(df_path, index_col=0)
df_filtered.index = df_filtered.index.str.split('.').str[0]
gene_rdc = df_filtered["Ribosome_Density_Change"].dropna()

In [28]:
df_filtered

,Chr,Start,End,Strand,Length,CLIP_35L33G,CLIP-let7g.bam,RNA_control,RNA_siLin28a,RNA_siLuc,RPF_siLin28a,RPF_siLuc,Normalized_CLIP,Normalized_RNA_siLuc_for_CLIP,CLIP_Enrichment,TE_raw_siLin28a,TE_raw_siLuc,TE_norm_siLin28a,TE_norm_siLuc,Ribosome_Density_Change
Geneid,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000033845,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4843429;4843434;4844659;4846600;4847748;484774...,4844739;4847024;4844739;4847024;4847871;484787...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,8487,4014,0,1355,3022,2563,1859,3191,0.000295,0.000263,0.163413,0.615283,1.244930,0.709876,0.938470,-0.402742
ENSMUSG00000033813,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4928037;4928261;4937693;4937693;4948250;494825...,4928199;4928726;4937755;4937755;4948355;494835...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+,3017,1658,0,762,2130,2225,328,609,0.000122,0.000228,-0.907731,0.154388,0.274034,0.178123,0.206576,-0.213798
ENSMUSG00000033793,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,5140241;5153303;5153333;5153347;5153355;515335...,5140367;5153501;5153848;5153501;5153501;515350...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;...,9746,1111,0,474,1260,1031,603,1128,0.000082,0.000106,-0.375877,0.478985,1.093992,0.552624,0.824688,-0.577550
ENSMUSG00000025907,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,6276421;6276451;6284869;6284887;6285232;629382...,6276494;6276494;6285181;6285181;6285476;629396...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;...,14690,1520,0,280,694,903,132,251,0.000112,0.000093,0.267035,0.191367,0.278761,0.220788,0.210139,0.071313
ENSMUSG00000051285,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,7159144;7159154;7159328;7159390;7190418;721786...,7159440;7159440;7159440;7159440;7190839;721796...,+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;+;...,6654,3508,0,529,2360,1775,208,138,0.000257,0.000182,0.498837,0.088522,0.078266,0.102131,0.058999,0.791652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000064366,chrM,11671,11741,+,71,97,0,19,43,44,2508,2021,0.000007,0.000005,0.639266,57.022727,44.933333,65.789387,33.872244,0.957751
ENSMUSG00000064367,chrM,11742,13565,+,1824,3665,0,5281,15549,17408,5986,5305,0.000269,0.001786,-2.731146,0.385016,0.304785,0.444208,0.229757,0.951128
ENSMUSG00000064368,chrM,13552,14070,-,519,1666,0,1976,6741,8052,11471,10842,0.000122,0.000826,-2.755863,1.701572,1.346455,1.963172,1.015002,0.951703


In [29]:
gene_rdc

Geneid
ENSMUSG00000033845   -0.402742
ENSMUSG00000033813   -0.213798
ENSMUSG00000033793   -0.577550
ENSMUSG00000025907    0.071313
ENSMUSG00000051285    0.791652
                        ...   
ENSMUSG00000064366    0.957751
ENSMUSG00000064367    0.951128
ENSMUSG00000064368    0.951703
ENSMUSG00000064370    0.790696
ENSMUSG00000064372    0.313233
Name: Ribosome_Density_Change, Length: 8106, dtype: float64

In [ ]:
# === Step 2: Ensembl → UniProt 매핑 ===
target_ensembl_ids = set(df_filtered.index)
ensembl_to_uniprot = {}

with open(idmap_path, "rt") as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 3 and parts[1] == "Ensembl":
            ensembl_id_raw = parts[2]
            ensembl_id = ensembl_id_raw.split('.')[0]  # ✅ 버전 제거
            if ensembl_id in target_ensembl_ids:
                ensembl_to_uniprot[ensembl_id] = parts[0]


print(f"총 매핑된 Ensembl ID 수: {len(ensembl_to_uniprot)}")

In [ ]:
# 총 매핑된 Ensembl ID 수: 7946

In [ ]:
# 저장 (옵션)
df_map = pd.DataFrame(list(ensembl_to_uniprot.items()), columns=["Ensembl_ID", "UniProt_ID"])
df_map.to_csv("/Users/jeongmin/Downloads/ensembl_to_uniprot.csv", index=False)

In [ ]:
# linux로 GO term이 하나라도 존재하는 줄만 미리 뽑아서 저장
# 마우스(Mus musculus, taxon:10090)만 미리 필터링
# gzcat /Users/jeongmin/Downloads/goa_uniprot_all.gaf.gz | grep -v '^!' | grep 'GO:' | grep 'taxon:10090' > /Users/jeongmin/Downloads/goa_mouse_filtered.gaf

In [ ]:
# === Step 3: UniProt → GO term 매핑 ===
uniprot_to_go = defaultdict(set)

with open(goa_path, "rt") as f:  # gzip 아님
    for line in f:
        if line.startswith("!"):
            continue
        parts = line.strip().split('\t')
        if len(parts) >= 5:
            uniprot_id = parts[1]
            go_term = parts[4]
            uniprot_to_go[uniprot_id].add(go_term)

In [ ]:
# 저장 (옵션)
rows = [
    {"UniProt_ID": uid, "GO_Terms": ";".join(sorted(go_set))}
    for uid, go_set in uniprot_to_go.items()
]

df_go_map = pd.DataFrame(rows)
df_go_map.to_csv("/Users/jeongmin/Downloads/uniprot_to_go.csv", index=False)

In [ ]:
# === Step 4: GO term → Ensembl Gene ID 매핑 구성 ===
go_to_genes = defaultdict(set)
for ens_id in gene_rdc.index:
    uniprot_id = ensembl_to_uniprot.get(ens_id)
    if uniprot_id:
        for go_term in uniprot_to_go.get(uniprot_id, []):
            go_to_genes[go_term].add(ens_id)

In [ ]:
print(f"GO term 개수: {len(go_to_genes)}")

In [ ]:
# GO term 개수: 9079

In [ ]:
for i, (go, genes) in enumerate(go_to_genes.items()):
    print(f"{go} → {len(genes)} genes")
    if i >= 9:
        break  # 상위 10개만 보기

GO:0003676 → 265 genes
GO:0046872 → 1047 genes
GO:0003677 → 532 genes
GO:0005634 → 1943 genes
GO:0006368 → 15 genes
GO:0008270 → 616 genes
GO:0006351 → 39 genes
GO:1902600 → 58 genes
GO:0046961 → 16 genes
GO:0000221 → 4 genes

In [ ]:
from scipy.stats import mannwhitneyu
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# === Step 5: Mann-Whitney U Test ===
results = []
for go_term, genes in go_to_genes.items():
    if len(genes) < 5:
        continue
    in_group = gene_rdc.loc[gene_rdc.index.isin(genes)]
    out_group = gene_rdc.loc[~gene_rdc.index.isin(genes)]
    if len(in_group) > 0 and len(out_group) > 0:
        stat, pval = mannwhitneyu(
            in_group,
            out_group,
            alternative='two-sided',
            method='asymptotic'
        )
        results.append((go_term, len(genes), in_group.mean(), pval))


In [ ]:
for row in results[:5]:
    print(f"GO: {row[0]}, N={row[1]}, Mean RDC={row[2]:.3f}, p={row[3]:.4e}")

GO: GO:0003676, N=265, Mean RDC=-0.625, p=4.3919e-26
GO: GO:0005634, N=1943, Mean RDC=-0.449, p=1.8117e-63
GO: GO:0006368, N=15, Mean RDC=-0.749, p=2.2923e-03
GO: GO:0003677, N=532, Mean RDC=-0.469, p=7.0196e-19
GO: GO:0008270, N=616, Mean RDC=-0.422, p=4.1979e-14

In [ ]:
# === Step 6: FDR 보정 및 results_df 생성 ===
results_df = pd.DataFrame(results, columns=["GO_term", "n_genes", "mean_rdc", "pval"])
results_df["fdr"] = smm.multipletests(results_df["pval"], method="fdr_bh")[1]
results_df["-log10(FDR)"] = -np.log10(results_df["fdr"])

In [ ]:
# === CLIP enrichment 값 계산 및 추가 ===
clip_enrichment_dict = {
    go_term: df_filtered.loc[df_filtered.index.isin(genes), "CLIP_Enrichment"].dropna().mean()
    for go_term, genes in go_to_genes.items()
}
results_df["clip_enrichment"] = results_df["GO_term"].map(clip_enrichment_dict)

In [ ]:
# === 유의한 GO term 필터링 ===
sig_results = results_df[results_df["fdr"] < 0.05].copy()
sig_results.to_csv("/Users/jeongmin/Downloads/sig_results_GO_enrichment.csv", index=False)

In [ ]:
print(sig_results.sort_values("fdr").head(5))

 GO_term  n_genes  mean_rdc  ...            fdr  -log10(FDR)  clip_enrichment
90   GO:0016020     1789  0.153882  ...  2.889719e-237   236.539144        -0.128158
19   GO:0005783      460  0.200246  ...   4.606095e-62    61.336667         0.109618
1    GO:0005634     1943 -0.448916  ...   1.132890e-60    59.945812        -0.793606
70   GO:0005789      310  0.266367  ...   7.894996e-55    54.102648         0.166805
103  GO:0005886      669  0.042771  ...   6.211669e-39    38.206792        -0.134128

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm

# === 저장된 파일 불러오기 ===
sig_results = pd.read_csv("/Users/jeongmin/Downloads/sig_results_GO_enrichment.csv")


In [ ]:
from goatools.obo_parser import GODag

# OBO 파일 로드
obo_path = "/Users/jeongmin/Downloads/go-basic.obo"
go_dag = GODag(obo_path)

# 예시: 특정 GO ID의 이름 확인
print(go_dag["GO:0005737"].name)  # "mitochondrion"


cytoplasm

In [ ]:
# 고유 GO term 리스트에서 이름 추출
go_name_dict = {go: go_dag[go].name for go in sig_results["GO_term"] if go in go_dag}

# 이름 추가
sig_results["GO_name"] = sig_results["GO_term"].map(go_name_dict)

# 주석 텍스트 만들기
sig_results["GO_label"] = sig_results.apply(
    lambda row: f"{row['GO_name']} ({row['n_genes']})\nC={row['pval']:.1e} R={row['fdr']:.1e}",
    axis=1
)


In [ ]:
import pandas as pd
import IPython.display as disp

disp.display(top_labels[["GO_term", "GO_name", "clip_enrichment", "mean_rdc", "n_genes", "fdr"]])


      GO_term                         GO_name  clip_enrichment  mean_rdc  n_genes            fdr
40   GO:0016020                        membrane        -0.128158  0.153882     1789  2.889719e-237
10   GO:0005783           endoplasmic reticulum         0.109618  0.200246      460   4.606095e-62
1    GO:0005634                         nucleus        -0.793606 -0.448916     1943   1.132890e-60
31   GO:0005789  endoplasmic reticulum membrane         0.166805  0.266367      310   7.894996e-55
48   GO:0005886                 plasma membrane        -0.134128  0.042771      669   6.211669e-39
14   GO:0005737                       cytoplasm        -0.836202 -0.422874     1771   9.960975e-39
123  GO:0005576            extracellular region        -0.060378  0.280829      184   2.740796e-32
65   GO:0055085         transmembrane transport         0.144456  0.351946      111   8.462988e-24
0    GO:0003676            nucleic acid binding        -0.810347 -0.624921      265   9.154610e-24
153  GO:0005615             extracellular space        -0.044841  0.334908      111   1.907743e-23